# SPARQL Workshop - Section 02: Traversing the graph

Now that you have the basics down, let's start working on queries that are more related to traversing a graph structure.  In the following sections. You will use the SPARQL queries necessary for navigating a graph.  This includes crossing common edges and related vertices along a given traversal path.

## Graph Model

As we have in previous notebooks, we've included here a copy of the graph data model for this dataset as reference to aid in writing your queries.

<img src="https://ee-assets-prod-us-east-1.s3.amazonaws.com/modules/f3f89ef4607743429fb01ae23d983197/v1/workshop/images/datamodel-rdf-updated.png"/>



## Following Edges

Storing relationships in the same manner as entities is a key differentiating factor for graph databases.  Moreso, traversing these relationships is what make graph databases powerful and unique to fulfill the role of a "purpose-built" tool for querying highly connected datasets.  In the following exercises, we will use SPARQL queries to show how to traverse across relationships, or edges, in a graph.

### 02.01 - Simple Edge Traversals

Let's start with a simple traversal that crosses an edge.  In the depiction of the data model above, you will notice that Artists are connected to different productions (i.e. movies, tvEpisodes, etc.) via edges that are labeled with the different credits or roles.  If we wanted to find all productions that a given Artist worked on, we can traverse across an edge with a given credit/role to the productions related to that Artist.  Let's give this a try.  

In the following example, we will find the movies directed by Christopher Nolan. In this example, we start by finding the resource representative of Christopher Nolan using the pattern `?artist dataProp:name "Christopher Nolan"` and then we use the `?production objProp:director ?artist` pattern to traverse across the incoming edge with the predicate of _objProp:director_.  At this point, we now have a list of all of the vertices related to productions of which Christopher Nolan directed.  We then use the `?production dataProp:title ?title` predicate to list out the titles of each of these productions.



In [ ]:
%%sparql

PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>

SELECT ?title
WHERE 
{
    ?artist dataProp:name "Christopher Nolan" .
    ?production objProp:director ?artist .
    ?production dataProp:title ?title
} 

### 02.02 - Counting Items with COUNT

Similar to doing `COUNT()` in SQL, you can also do a `COUNT()` in SPARQL.  The only difference being that you'll need to associate an output name to column associated with the count output.

In the following example, we count how many productions have a genre of 'Sci-Fi':

In [ ]:
%%sparql

PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>

SELECT (COUNT(?production) as ?prodCount) 
WHERE 
{
    ?genre rdf:type class:Genre .
    ?genre dataProp:genre "Sci-Fi" .
    ?production objProp:genre ?genre
}

Looking at the query above in more detail:

- The `?genre rdf:type class:Genre` pattern denotes that we want to start by looking at the resources of type Genre
- Next, the `?genre dataProp:genre "Sci-Fi"` pattern states that we want to only look at the Genre with the genre property value of "Sci-Fi".
- Lastly, the `?production objProp:genre ?genre` pattern is the "edge traversal" in this case. It states that we want to traverse from the "Sci-Fi" Genre vertex to the related Production (movie, tvSeries, etc.) vertices.

### 02.03 - Multiple Hop Traversals & DISTINCT

Here's an example where you can chain multiple "edge crossing" patterns together to navigate deeper into a graph.  In the following example, find the Person with ID 'person378' and count the number of unique people (themselves _included_) who follow the same Artists.

You will notice that we're also using another SPARQL command here, `DISTINCT`.  Much like `COUNT`, `DISTINCT` does what it implies.  It removes duplicates and returns a list of de-duplicated bindings for the selected variable.  We need `DISTINCT` in this case as each Artist vertex that we traverse across may have corresponding Follow edge connections to some of the same people.  We don't care about the duplicates here, as we only want a count of the unique Person URIs that follow the same Artists as Person vertex 'person378'.



In [ ]:
%%sparql

PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>
PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>

SELECT (COUNT(DISTINCT ?people) as ?peopleCount) 
WHERE 
{
    resource:person378 objProp:follows ?artists .
    ?people objProp:follows ?artists
}

### 02.04 - Traversing Multiple Edge Types (predicates)

There are certain times where you need to traverse the graph across multiple edges.  In our IMDb data model, we have separate edge/relationship types for different production roles (i.e. actor, actress, director, etc.).  What if we wanted to traverse multiple edge types to get to our results.
 
As an example, say we wanted to output all artist (actors and actresses) that co-starred with Halle Berry and the movies in which they appeared together. Let's try that via the following query:

In [ ]:
%%sparql

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>

SELECT DISTINCT ?title ?costarname
WHERE 
{
    ?halle dataProp:name "Halle Berry" . 
    ?movies rdf:type class:Movie .
    ?movies objProp:actress ?halle .
    ?movies objProp:actor|objProp:actress ?costars .
    ?movies dataProp:title ?title .
    ?costars dataProp:name ?costarname
}

### 02.05 - Using FILTER to filter results

That last query was pretty cool, but it did include Halle in the results.  How can we get results without Halle listed and only her costars?  We can do this by adding a conditional and using the `FILTER` step.  The `FILTER` does as it suggests and allows us to provide a conditional.  In this case, we can say "return all of these results where the costar does not equal 'Halle Berry'".

In [ ]:
%%sparql

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>

SELECT DISTINCT ?title ?costarname WHERE {
    ?halle dataProp:name "Halle Berry" . 
    ?movies rdf:type class:Movie .
    ?movies objProp:actress ?halle .
    ?movies objProp:actor|objProp:actress ?costars .
    ?movies dataProp:title ?title .
    ?costars dataProp:name ?costarname .
    FILTER (?costars != ?halle)
}

### 02.07 - Aggregate Results and using GROUP BY and ORDER BY

Let's trying something a bit more complex.  Ever wondered how Facebook, Twitter, or LinkedIn provide you with a list of people that you "might want to connect with"?  This is that type of query in its simplist form.  This query can also be referred to as the __Friends of Friends__ (FoF) query. 

We can simulate FoF in our IMDb dataset by looking at co-stars of co-stars for a particular performer.  We can basically ask "what performers should our unique performer look to star with based on their total number of common co-stars"

In doing this type of query, we'll need to leverage different intermediate variables for aggregations.  In the query below, you'll notice that we have both `?productions1` and `?productions2` along with `?costars1` and `?costars2`.  These intermediate aggregations can then be used for comparisons/filters later in our traversal. In this case, we'll want to aggregate all co-stars of a given performer, then the co-stars of co-stars, and then count the number of times the co-stars of co-stars appear across common co-star connections.  We will also want to filter out co-stars and our given performer from the results.

In the following example, we'll use Rebel Wilson and find which performer she has not appeared with but shares the highest number of common co-stars.


In [ ]:
%%sparql

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>

SELECT ?name (COUNT(?name) as ?count) WHERE {
    ?rebel dataProp:name "Rebel Wilson" .
    ?rebel rdf:type class:Artist .
    ?productions1 objProp:actress ?rebel .
    ?productions1 objProp:actor|objProp:actress ?costars1 .
    ?productions2 objProp:actor|objProp:actress ?costars1 .
    ?productions2 objProp:actor|objProp:actress ?costars2 .
    ?costars2 dataProp:name ?name .
    FILTER(?costars2 != ?rebel && ?costars2 != ?costars1) .
} GROUP BY ?name ORDER BY DESC(?count)

### Review

In the previous notebook we started using SPARQL to traverse across the RDF graph.  We displayed how these queries can be used to take advantage of the first class representation of relationships in a graph.  Lastly, we also looked at different types of aggregations and how to display results in an ordered fashion.  We will continue building on the use of these patterns in the next notebook.


### End of Section 02 - Move onto the next notebook.